In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-11 08:01:46,153 utils 400 [INFO]    [logger_func] start 


In [225]:
#========================================================================
# CV CHECKER 
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]
base_idx = train.reset_index()[[key, target]]

check_model_path = '../stack/0208_002_stack_lgb_lr0.01_235feats_multi157_valterm_5seed_4leaves_iter1031_TERM24_CV3-631440687534257_LB.gz'
check_model = utils.read_pkl_gzip(check_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
train = train.join(check_model)

y_train = train[target].values
check_pred = train['pred_mean'].values
check_score = np.sqrt(mean_squared_error(y_train, check_pred))
print(check_score)
#========================================================================


100%|██████████| 3/3 [00:00<00:00, 56.44it/s]

3.7031587732980125


In [5]:
#========================================================================
# Corr Checker
from itertools import combinations
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
ens_list = glob.glob('../ensemble/*.gz')
combi_path = combinations(ens_list, 2)

# 相関
for p_1, p_2 in combi_path:
    m1 = utils.read_pkl_gzip(p_1)[[key, 'pred_mean']].set_index(key)
    m2 = utils.read_pkl_gzip(p_2)[[key, 'pred_mean']].set_index(key)
    m1['p2'] = m2['pred_mean']
    cv1 = re.search(r'CV([^/.]*)_LB.gz', p_1).group(1)
    cv2 = re.search(r'CV([^/.]*)_LB.gz', p_2).group(1)
    corr = np.corrcoef(m1['pred_mean'], m1['p2'].values).min()
    logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 46.26it/s]
2019-02-10 17:21:35,509 utils 16 [INFO]    [<module>] CORR: 0.9697785847901149 | CV3-6439 vs CV3-6437 
2019-02-10 17:21:36,322 utils 16 [INFO]    [<module>] CORR: 0.9500074357544697 | CV3-6439 vs CV3-6215 
2019-02-10 17:21:36,988 utils 16 [INFO]    [<module>] CORR: 0.9596269179540176 | CV3-6439 vs CV3-6406 
2019-02-10 17:21:37,800 utils 16 [INFO]    [<module>] CORR: 0.9511834437402763 | CV3-6437 vs CV3-6215 
2019-02-10 17:21:38,599 utils 16 [INFO]    [<module>] CORR: 0.9523197432194437 | CV3-6437 vs CV3-6406 
2019-02-10 17:21:39,544 utils 16 [INFO]    [<module>] CORR: 0.9617856854149184 | CV3-6215 vs CV3-6406 


In [2]:
# Data Load
# Model Load
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
ens_list = glob.glob('../ensemble/*.gz')
for path in ens_list:
    try:
        cv = re.search(r'CV([^/.]*)_LB.gz', path).group(1)
    except AttributeError:
        cv = re.search(r'CV([^/.]*).gz', path).group(1)
    
    try:
        blend = utils.read_df_pkl(path).set_index(key)['pred_mean']
    except KeyError:
        blend = utils.read_df_pkl(path).set_index(key)['prediction']
    base[f'base_{cv}'] = blend
#========================================================================

out_list = glob.glob('../out_ensemble/*.gz')
out_list = ['../out_ensemble/0208_215_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz']
for path in out_list:
    cv = re.search(r'CV([^/.]*)_LB.gz', path).group(1)
    blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
    base[f"out_{cv}"] = blend

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf['clf_pred'] =  clf['pred_mean']
#========================================================================

# Indexをそろえる
base['clf_pred'] = clf['clf_pred']
base.head()

100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


,target,base_3-643944732150466,base_3-6745325015227253,base_3-7005383169300727,base_3-6437143344456357,base_3-7421434632512685,base_3-6215750935280235,base_3-6406114043915188,out_8-413966707443706,clf_pred
card_id,,,,,,,,,,
C_ID_92a2005557,-0.820283,-0.275469,-0.442548,-0.440412,-0.189379,-0.386080,-0.348066,-0.308465,-0.349654,0.000444
C_ID_3d0044924f,0.392913,-0.512262,0.526564,0.191130,-0.276453,0.641302,-0.433948,-0.501848,0.261769,0.007831
C_ID_d639edf6cd,0.688056,0.615076,0.499075,0.316775,0.586058,0.691890,0.707558,0.570250,0.725757,0.004074
C_ID_186d6a6901,0.142495,0.029858,0.316660,0.343618,0.128617,0.301038,0.160033,0.129957,0.175317,0.000797
C_ID_cdbd2c0db2,-0.159749,-0.045465,-0.142978,-0.133101,-0.008663,-0.081131,-0.110775,-0.293786,-0.239419,0.000251


In [3]:
base_cols = [col for col in base.columns if col.count('base_')]
out_cols = [col for col in base.columns if col.count('out_')]
base_cols = sorted(base_cols)
base = base[base_cols + [target, 'clf_pred'] + out_cols]
base.head()

,base_3-6215750935280235,base_3-6406114043915188,base_3-6437143344456357,base_3-643944732150466,base_3-6745325015227253,base_3-7005383169300727,base_3-7421434632512685,target,clf_pred,out_8-413966707443706
card_id,,,,,,,,,,
C_ID_92a2005557,-0.348066,-0.308465,-0.189379,-0.275469,-0.442548,-0.440412,-0.386080,-0.820283,0.000444,-0.349654
C_ID_3d0044924f,-0.433948,-0.501848,-0.276453,-0.512262,0.526564,0.191130,0.641302,0.392913,0.007831,0.261769
C_ID_d639edf6cd,0.707558,0.570250,0.586058,0.615076,0.499075,0.316775,0.691890,0.688056,0.004074,0.725757
C_ID_186d6a6901,0.160033,0.129957,0.128617,0.029858,0.316660,0.343618,0.301038,0.142495,0.000797,0.175317
C_ID_cdbd2c0db2,-0.110775,-0.293786,-0.008663,-0.045465,-0.142978,-0.133101,-0.081131,-0.159749,0.000251,-0.239419


In [11]:
cp = 'clf_pred'
base[base[cp]>0.1]

,base_3-6215750935280235,base_3-6406114043915188,base_3-6437143344456357,base_3-643944732150466,base_3-6745325015227253,base_3-7005383169300727,base_3-7421434632512685,target,clf_pred,out_8-413966707443706
card_id,,,,,,,,,,
C_ID_929cf67c44,-17.546794,-15.986625,-18.407963,-17.180591,-0.494749,-3.111159,-0.997317,-0.687414,0.478797,-1.817747
C_ID_365e57087b,-14.211813,-13.442176,-15.617129,-17.063731,-1.933665,-3.814056,-1.461416,-33.219281,0.499032,-1.516319
C_ID_d1812da0cb,-15.391655,-14.063268,-15.584064,-16.844658,-0.072128,0.115985,-1.576233,0.681776,0.507379,-0.036733
C_ID_80e8a2c964,-19.605306,-14.175204,-20.353089,-15.921943,0.594143,-1.611595,-0.039009,-33.219281,0.581903,0.945188
C_ID_775bd28f7b,-19.192759,-21.091663,-14.134903,-15.778664,-0.110080,-1.205721,-0.882730,-33.219281,0.556185,0.607890
C_ID_b6f7db5880,-16.942016,-12.718003,-15.567457,-14.646756,-2.602578,-1.519958,-0.527336,-33.219281,0.596424,0.121720
C_ID_981df8bb2b,-16.316571,-13.993532,-13.721415,-15.597467,-4.710669,-2.964472,-3.058343,-1.596857,0.471793,-0.300613
C_ID_26e9fcf59c,-17.589418,-13.406814,-15.415103,-13.118986,-5.466681,-7.378551,-3.316444,-5.870283,0.510774,0.387309
C_ID_0b358310f5,-19.514482,-18.411255,-18.301125,-18.012158,-7.026054,-5.107373,-6.020628,-33.219281,0.567377,-1.431637


In [163]:
#========================================================================
# Simple Blender 
min_thres = 0.0072
pred = 'prediction'

base[pred] = base[base_cols[0]] * 0.3 + base[base_cols[1]] * 0.15 + base[base_cols[2]] * 0.15 + base[base_cols[3]] * 0.15 + base[base_cols[4]] * 0.15 + base[base_cols[5]] * 0.05 + base[base_cols[6]] * 0.05
base[pred] = base[base_cols[0]] * 0.5 + base[base_cols[1]] * 0.1 + base[base_cols[2]] * 0.1 + base[base_cols[3]] * 0.10 + base[base_cols[4]] * 0.2
# base[pred] = base[base_cols[0]] * 1.0 + base[base_cols[4]] * 1.0
# base[pred] = base[base_cols[4]] * 1.5
base.loc[base['clf_pred']< min_thres, pred] = base.loc[base['clf_pred']< min_thres, out_cols[0]]
# base.loc[base['clf_pred']>=min_thres, pred] = base.loc[base['clf_pred']>=min_thres, pred]

train = base[~base[target].isnull()]
y_pred = train[pred].values
y_train = train[target].values
score = np.sqrt(mean_squared_error(y_train, y_pred))
print(score)

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{len(base_cols)}_out_blend{len(out_cols)}_minthres{min_thres}_CV{score}_LB')
display(submit.head())
#========================================================================

3.607750314545443


,target
card_id,
C_ID_0ab67a22ab,-1.455610
C_ID_130fd0cbdd,-0.171461
C_ID_b709037bc5,-1.243021
C_ID_d27d835a9f,-0.059027
C_ID_2b5e3df5c2,-1.361652


In [36]:
from itertools import product, permutations
#========================================================================
# Blendのスコアを見る
#========================================================================
score_list = []
best_ratio_list = []
min_score = 100
train = base[~base[target].isnull()]
y_train = train[target].values


cnt = 0
while cnt!=10000:
    cnt+=1
    np.random.seed(cnt)
    ratio_list = np.arange(0.05, 1.0, 0.05)
    np.random.seed(cnt+1); np.random.shuffle(ratio_list)
    r1 = ratio_list.copy()
    np.random.seed(cnt*2); np.random.shuffle(ratio_list)
    r2 = ratio_list.copy()
    np.random.seed(cnt*3); np.random.shuffle(ratio_list)
    r3 = ratio_list.copy()
    np.random.seed(cnt*4); np.random.shuffle(ratio_list)
    r4 = ratio_list.copy()
    ratio_combi = zip(r1, r2, r3, r4)
    
    for i, ratio in enumerate(ratio_combi):
        
        p1 = train[base_cols[0]].values
        p2 = train[base_cols[1]].values
        p3 = train[base_cols[2]].values
        p4 = train[base_cols[3]].values
#         if cnt==1:
#             print(
#                 np.sqrt(mean_squared_error(y_train, p1))
#                 ,np.sqrt(mean_squared_error(y_train, p2))
#                 ,np.sqrt(mean_squared_error(y_train, p3))
#                 ,np.sqrt(mean_squared_error(y_train, p4))
#             )
#             sys.exit()
        
        ratio = np.array(ratio)
        r_sum = ratio.sum()
        ratio /= r_sum
        a = ratio[0]
        b = ratio[1]
        c = ratio[2]
        d = ratio[3]
        coef_list = [a,b,c,d]
        
        combi_list = permutations(coef_list)
        for combi in combi_list:
            e = combi[0]
            f = combi[1]
            g = combi[2]
            h = combi[3]
            y_pred = p1*e + p2*f + p3*g + p4*h
            
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                min_ratio = combi
                
                score_list.append(min_score)
                best_ratio_list.append(min_ratio)
        
    if cnt%200==0:
        print(f"BEST: {np.min(score_list)} | BEST RATIO: {best_ratio_list[np.argmin(score_list)]}")
        
print("Complete!!")

BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454

KeyboardInterrupt: 

In [4]:
#========================================================================
# Manual PostProcessing
# Setting
# upper_1. Best Score: 3.6114026721733525 | Best Manual: 25.319999999999993 | Max Thres: 0.511
# upper_2. Best Score: 3.6105940647875325 | Best Manual: 9.119999999999997 | Max Thres: 0.471
#========================================================================

is_manual = True
score_list = []
ratio_list = []
min_score = 100
pred = 'prediction'

train = base[~base[target].isnull()]
y_train = train[target].values
train[pred] = train['pred_mean']
train[pred] = train['no_out_pred']
min_thres = 0.0072
upper_thres = 0.511
upper_value = -25.32
# for min_thres in np.arange(0.005, 0.01, 0.0002):
for max_thres in np.arange(0.47, 0.48, 0.001):
    
    length = train.loc[train['clf_pred']> min_thres].shape[0]
    
    # Under PostProcessing
    train.loc[train['clf_pred']< min_thres, pred] = train.loc[train['clf_pred']< min_thres, 'no_out_pred']
    train.loc[train['clf_pred']>=min_thres, pred] = train.loc[train['clf_pred']>=min_thres, 'pred_mean']
    train.loc[train['no_out_flg']==1, pred] =  train.loc[train['no_out_flg']==1, 'no_out_pred']
     
    tmp_pred = train[pred].values

    # 閾値以上のtargetをマニュアル設定する
    if is_manual:
        for manual in np.arange(9.0, 10, 0.02):
            train[pred] = tmp_pred.copy()
            train.loc[train['clf_pred']> max_thres, pred] = manual*-1
            # Upper PosrProcessing
            train.loc[train['clf_pred']> upper_thres, pred] = upper_value
    
            y_pred = train[pred].values
        
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                best_thres = max_thres
                best_value = manual
                score_list.append(min_score)
                ratio_list.append(best_thres)
            if score<3.611:
#                 print(f"Best Score: {min_score} | Min Thres: {best_thres} | Max Thres: {max_thres} | MANUAL: {-1*manual}")
                print(f"Score: {score} | Best Score: {min_score} | Best Manual: {best_value} | Max Thres: {best_thres}")
        
    else:
        y_pred = train[pred].values
    
        score = np.sqrt(mean_squared_error(y_train, y_pred))
        if score<min_score:
            min_score = score
            best_thres = min_thres
            score_list.append(min_score)
            ratio_list.append(best_thres)
        print(f"Best Score: {min_score} | Best Thres: {best_thres} | LENGTH: {length}")

Score: 3.6107862840993854 | Best Score: 3.6107862840993854 | Best Manual: 9.0 | Max Thres: 0.47
Score: 3.610785562664395 | Best Score: 3.610785562664395 | Best Manual: 9.02 | Max Thres: 0.47
Score: 3.610784851653373 | Best Score: 3.610784851653373 | Best Manual: 9.04 | Max Thres: 0.47
Score: 3.610784151066326 | Best Score: 3.610784151066326 | Best Manual: 9.059999999999999 | Max Thres: 0.47
Score: 3.6107834609032596 | Best Score: 3.6107834609032596 | Best Manual: 9.079999999999998 | Max Thres: 0.47
Score: 3.6107827811641795 | Best Score: 3.6107827811641795 | Best Manual: 9.099999999999998 | Max Thres: 0.47
Score: 3.6107821118490917 | Best Score: 3.6107821118490917 | Best Manual: 9.119999999999997 | Max Thres: 0.47
Score: 3.610781452958003 | Best Score: 3.610781452958003 | Best Manual: 9.139999999999997 | Max Thres: 0.47
Score: 3.6107808044909184 | Best Score: 3.6107808044909184 | Best Manual: 9.159999999999997 | Max Thres: 0.47
Score: 3.6107801664478436 | Best Score: 3.6107801664478436

In [15]:
blend_num = 3
out_blend_num = 2
thres = 0.0072
base.loc[base['clf_pred']< thres, pred] = base.loc[base['clf_pred']< thres, 'no_out_pred']
base.loc[base['clf_pred']>=thres, pred] = base.loc[base['clf_pred']>=thres, 'pred_mean']
base.loc[base['no_out_flg']==1, pred] =  base.loc[base['no_out_flg']==1, 'no_out_pred']
display(base.head())

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{blend_num}_0.4_0.3_0.3_out_blend{out_blend_num}_0.7_0.3_CV{min_score}_LB')
display(submit.head())

,target,no_out_flg,pred_mean,p1,p2,p3,no_out_pred,clf_pred,prediction
card_id,,,,,,,,,
C_ID_92a2005557,-0.820283,NaN,-0.312293,-0.277248,-0.299641,-0.295437,-0.339001,0.000444,-0.339001
C_ID_3d0044924f,0.392913,NaN,-0.402629,-0.464311,-0.299189,-0.659761,0.276268,0.007831,-0.402629
C_ID_d639edf6cd,0.688056,NaN,0.740270,0.764349,0.759806,0.823495,0.770552,0.004074,0.770552
C_ID_186d6a6901,0.142495,NaN,0.134626,0.101346,0.134031,0.057771,0.170184,0.000797,0.170184
C_ID_cdbd2c0db2,-0.159749,1.0,-0.184077,-0.222948,-0.242942,-0.158849,-0.238870,0.000251,-0.238870


,target
card_id,
C_ID_0ab67a22ab,-1.659109
C_ID_130fd0cbdd,-0.203953
C_ID_b709037bc5,-1.298058
C_ID_d27d835a9f,-0.052482
C_ID_2b5e3df5c2,-1.371860


In [6]:
#========================================================================
# Stack同士の相関をみる
#========================================================================

corr_list = []
corr_path_list = []
blend_path_list = glob.glob('../stack/*LB.gz')
blend_path_list = ['../stack/0209_200_stack_lgb_lr0.01_297feats_1seed_71leaves_iter1450_OUT29.8580_CV3-6236254858483243_LB.gz']

# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)

best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
base = base.join(best_model)

Y = base[target].values
best_pred = base['pred_mean'].values
# base.drop([target, 'pred_mean'], axis=1, inplace=True)
# best_score = np.sqrt(mean_squared_error(y_train, best_pred))
# print(f"Best Score: {best_score}")

for i, path in enumerate(blend_path_list):
    tmp = utils.read_pkl_gzip(path).set_index(key)
    if tmp.shape[0] != best_pred.shape[0]:
        continue
    if path.count('TERM'):
        continue
    corr_path_list.append(path)
    
    if 'pred_mean' in tmp.columns:
        base['p2'] = tmp['pred_mean']
    else:
        base['p2'] = tmp['prediction']
    del tmp
    gc.collect()
    
    corr = np.corrcoef(base['pred_mean'].values, base['p2'].values).min()
    
    corr_list.append(corr)
    print(f"Corr: {corr} | PATH: {path}")
    
print("Complete!!")
df_corr = pd.DataFrame([corr_path_list, corr_list]).T
df_corr.columns = ['path', 'corr']
df_corr.sort_values(by='corr', inplace=True)
df_corr['path'] = df_corr['path'].map(lambda x: re.search(r'CV([^/.]*).gz', x).group(1))
df_corr.head()

100%|██████████| 3/3 [00:00<00:00, 54.62it/s]


Corr: 0.9788335451194622 | PATH: ../stack/0209_200_stack_lgb_lr0.01_297feats_1seed_71leaves_iter1450_OUT29.8580_CV3-6236254858483243_LB.gz
Complete!!


,path,corr
0,3-6236254858483243_LB,0.978834


In [ ]:
df_corr.sort_values(by='path')

In [ ]:
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]

# Model Load
best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
out_model_path = '../stack/0208_210_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)
out_model = utils.read_pkl_gzip(out_model_path)[[key, 'pred_mean']].set_index(key)
best_model['no_out_pred'] = out_model['pred_mean']

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.6 + clf['pred_mean_2'].values*0.4
#========================================================================

# Indexをそろえる
train = train.join(best_model)
train['clf_pred'] = clf['clf_pred']
train.head()